# Scraping "Decathlon"

In [87]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time
from serde import serialize, deserialize
from serde.json import to_json, from_json
import pandas as pd
from dataclasses import dataclass

In [3]:
nav = webdriver.Chrome()

In [4]:
adresse = "https://www.decathlontravel.com/voyages/sport-rando-nature?gclid=Cj0KCQjw8eOLBhC1ARIsAOzx5cGfJP_HygFO_qlzeJG0KARJN657Ecyd7a3bGXh3OdxAGVnE14RFuREaAmPdEALw_wcB"
nav.get(adresse)
code = nav.page_source
soupe = BS(code)
# type(code)

## Capture des boutons

In [5]:
bouton = nav.find_element(By.ID, "didomi-notice-agree-button")
bouton.screenshot("bouton1.png")

True

In [6]:
bouton.click()

In [7]:
bouton2 = nav.find_element_by_link_text("Afficher plus de voyages")
bouton2.screenshot("bouton2.png")

C:\Users\arkan\AppData\Local\Temp/ipykernel_14928/962775747.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bouton2 = nav.find_element_by_link_text("Afficher plus de voyages")


True

## Recherche nombre d'item total

In [8]:
count = nav.find_elements_by_class_name("counter")
count_text = count[0].text
motif_count = re.compile('\d+')
# print(motif_count.findall(count_text))
count_num = int(motif_count.findall(count_text)[0])


C:\Users\arkan\AppData\Local\Temp/ipykernel_14928/3479058077.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  count = nav.find_elements_by_class_name("counter")


## Boucle pour afficher tout les voyages

In [9]:
click_count = count_num//6

In [10]:
for i in range(click_count):
    bouton2.click()
    time.sleep(5)

In [11]:
bouton2.click()

## Copie du code source en json

In [192]:
code_full = nav.page_source

str

In [12]:
with open("code_full_decat.json", "w") as fichier:
    fichier.write(to_json(code_full))

In [103]:
soupe_full = BS(code_full, "lxml")
cells_full = soupe_full.find_all( name= "article")
len(cells_full) == count_num

True

In [104]:
cells_full[1]

<article class="result travel-search-engine-result megalink">
<div class="head">
<div class="photo">
<img alt="Randonneuses sur la plage, côte Atlantique" src="https://photos.altai-travel.com/600x0/randonnee-sur-la-plage-cote-atlantique-15415.jpg" style="object-position: 37% 51%" title="Randonnée sur la plage, côte Atlantique © "/>
</div>
<div class="minprice">À partir de <strong>320 €</strong></div>
<ul class="tags">
<li class="tag">Best seller</li>
</ul>
</div>
<div class="overview">
<div class="destination">
                                                            Maroc / Côte Atlantique                                                        </div>
<h2 class="heading"><a class="link" href="/voyages/sport-rando-nature/voyage-cote-atlantique-et-essaouira/MESSA5" target="_blank">Côte atlantique et Essaouira</a></h2>
<ul class="features">
<li class="feature is-duration">
<svg class="icon"><use xlink:href="/front/svg/icons.svg?v=1640259831#icon-duree"></use></svg>
<span class="name">D

In [143]:
# Pourquoi il ne trouve pas le lieux .... 
motif = re.compile("title.*\"(.*)..\"|minprice.*strong.(.*)....strong|destinati(...)|value.*\".(.*)jours|value..(.*)..span|theme.*\"(.*)\"")

feat_val = list()
for i in range(len(cells_full)):
    texte_cell = str(cells_full[i])
    # texte_cell1 = texte_cell.replace("\n","")
    index = 0
    for j in motif.finditer(texte_cell):
        if index > 5:
            exit
        else:    
            c = j.groups()
            feat_val.append(c[index])
            index = index +1

feat_val

['Marcheurs dans le désert de Nesrate, Maroc ',
 '290',
 'on"',
 '4 ',
 'Découverte',
 'Rando nature',
 'Randonnée sur la plage, côte Atlantique ',
 '320',
 'on"',
 '5 ',
 'Découverte',
 'Rando nature',
 'Sugiton ',
 '400',
 'on"',
 '7 ',
 'Initié',
 'Rando nature',
 '',
 '450',
 'on"',
 '8 ',
 'Découverte',
 'Rando nature',
 'Ile de Serifos, dans les Cyclades en Grèce ',
 '580',
 'on"',
 '8 ',
 'Découverte',
 'Rando nature',
 "Belle Île en Mer lors d'une randonnée ",
 '580',
 'on"',
 '6 ',
 'Initié',
 'Rando nature',
 'Randonnée dans les Cinque Terre ',
 '715',
 'on"',
 '6 ',
 'Découverte',
 'Rando nature',
 'Canaries, randonnée à la Gomera ',
 '720',
 'on"',
 '15 ',
 'Découverte',
 'Rando nature',
 'Présentation du Sud Maroc aux enfants ',
 '895',
 'on"',
 '8 ',
 'Découverte',
 'Rando nature',
 'Vue sur les falaises de Moher ',
 '1\u202f000',
 'on"',
 '8 ',
 'Découverte',
 'Rando nature',
 'Ponta de São Lourenço falaises ',
 '1\u202f245',
 'on"',
 '8 ',
 'Initié',
 'Rando nature',
 '

In [49]:
# motif = re.compile("title.*\"(.*)..\"|minprice.*strong.(.*)....strong|value.*\".(.*)jours|value..(.*)..span|theme.*\"(.*)\"")

# feat_val = list()
# for i in range(len(cells_full)):
#     cell = cells_full[i].contents
#     texte_cell = str(cell)
#     # texte_cell1 = texte_cell.replace(" ","")
#     index = 0
#     for j in motif.finditer(texte_cell):
#         if index > 4:
#             exit
#         else:    
#             c = j.groups()
#             feat_val.append(c[index])
#             index = index +1

# feat_val

['Marcheurs dans le désert de Nesrate, Maroc ',
 '290',
 '4 ',
 'Découverte',
 'Rando nature',
 'Randonnée sur la plage, côte Atlantique ',
 '320',
 '5 ',
 'Découverte',
 'Rando nature',
 'Sugiton ',
 '400',
 '7 ',
 'Initié',
 'Rando nature',
 '',
 '450',
 '8 ',
 'Découverte',
 'Rando nature',
 'Ile de Serifos, dans les Cyclades en Grèce ',
 '580',
 '8 ',
 'Découverte',
 'Rando nature',
 "Belle Île en Mer lors d'une randonnée ",
 '580',
 '6 ',
 'Initié',
 'Rando nature',
 'Randonnée dans les Cinque Terre ',
 '715',
 '6 ',
 'Découverte',
 'Rando nature',
 'Canaries, randonnée à la Gomera ',
 '720',
 '15 ',
 'Découverte',
 'Rando nature',
 'Présentation du Sud Maroc aux enfants ',
 '895',
 '8 ',
 'Découverte',
 'Rando nature',
 'Vue sur les falaises de Moher ',
 '1\u202f000',
 '8 ',
 'Découverte',
 'Rando nature',
 'Ponta de São Lourenço falaises ',
 '1\u202f245',
 '8 ',
 'Initié',
 'Rando nature',
 'Randonneur au sommet du Teide à Ténérife ',
 '1\u202f315',
 '8 ',
 'Initié',
 'Rando nat

In [50]:
# for i in range(0,len(feat_val)):
#     if feat_val[i] == None:
#         print(i)


640
641
642
643


In [51]:
# del feat_val[640:644]

In [52]:
# feat_val

['Marcheurs dans le désert de Nesrate, Maroc ',
 '290',
 '4 ',
 'Découverte',
 'Rando nature',
 'Randonnée sur la plage, côte Atlantique ',
 '320',
 '5 ',
 'Découverte',
 'Rando nature',
 'Sugiton ',
 '400',
 '7 ',
 'Initié',
 'Rando nature',
 '',
 '450',
 '8 ',
 'Découverte',
 'Rando nature',
 'Ile de Serifos, dans les Cyclades en Grèce ',
 '580',
 '8 ',
 'Découverte',
 'Rando nature',
 "Belle Île en Mer lors d'une randonnée ",
 '580',
 '6 ',
 'Initié',
 'Rando nature',
 'Randonnée dans les Cinque Terre ',
 '715',
 '6 ',
 'Découverte',
 'Rando nature',
 'Canaries, randonnée à la Gomera ',
 '720',
 '15 ',
 'Découverte',
 'Rando nature',
 'Présentation du Sud Maroc aux enfants ',
 '895',
 '8 ',
 'Découverte',
 'Rando nature',
 'Vue sur les falaises de Moher ',
 '1\u202f000',
 '8 ',
 'Découverte',
 'Rando nature',
 'Ponta de São Lourenço falaises ',
 '1\u202f245',
 '8 ',
 'Initié',
 'Rando nature',
 'Randonneur au sommet du Teide à Ténérife ',
 '1\u202f315',
 '8 ',
 'Initié',
 'Rando nat

In [53]:
# feat_val2 = list( zip(*[iter(feat_val)]*5))

In [185]:
@serialize
@deserialize
@dataclass
class Sejour :
    description : str
    prix : int
    lieux : str
    duree : int
    diff : str
    theme : str 

In [153]:
texte = str(cells_full[13])
texte = texte.replace("\n", "")
texte
motif = re.compile("title(.*?)>")
titre = motif.findall(texte)[0]
titre

'="Randonnée dans le parc national du Teide, Tenerife. ©\xa0"/'

In [156]:
motif = re.compile("destination(.*?)h2")
lieu = motif.findall(texte)[0]
lieu.replace(" ", "")

'">Canaries/Espagne</div><'

In [158]:
motif = re.compile("minprice.*strong.(.*)....strong")
prix = motif.findall(texte)[0]
prix

'1\u202f445'

In [204]:
motif = re.compile("value.*\".(.*)jours")
duree = motif.findall(texte)[0]
duree = int(duree)
duree

15

In [168]:
motif = re.compile("Niveau.*value..(.*)..span")
diff = motif.findall(texte)[0]
diff

'Découverte'

In [183]:
motif = re.compile("theme.*tooltip..(.*?)\"")
theme = motif.findall(texte)[0]
theme


'Rando nature'

In [232]:
resultats = list()
for cell in cells_full:
    texte = str(cell)
    texte = texte.replace("\n", "")
    motif = re.compile("title(.*?)>")
    description = motif.findall(texte)[0][2:-5]
    motif = re.compile("destination(.*?)h2")
    lieu = motif.findall(texte)[0]
    lieu = lieu.replace(" ", "")[2:-7]
    motif = re.compile("minprice.*strong.(.*)....strong")
    prix = motif.findall(texte)[0]
    prix = int(prix.replace("\u202f",""))
    motif = re.compile("value.*\".(.*)jours.*span")
    duree = motif.findall(texte)[0]
    duree = int(duree)
    motif = re.compile("Niveau.*value..(.*)..span")
    diff = motif.findall(texte)[0]
    motif = re.compile("theme.*tooltip..(.*?)\"")
    theme = motif.findall(texte)[0]
    
    resultats.append(
        Sejour(
            description,
            prix,
            lieu,
            duree,
            diff,
            theme
        )
    )

In [233]:
resultats

[Sejour(description='Marcheurs dans le désert de Nesrate, Maroc', prix=290, lieux='Maroc/Désert', duree=4, diff='Découverte', theme='Rando nature'),
 Sejour(description='Randonnée sur la plage, côte Atlantique', prix=320, lieux='Maroc/CôteAtlantique', duree=5, diff='Découverte', theme='Rando nature'),
 Sejour(description='Sugiton', prix=400, lieux='Provence', duree=7, diff='Initié', theme='Rando nature'),
 Sejour(description='', prix=450, lieux='Madère/Portugal', duree=8, diff='Découverte', theme='Rando nature'),
 Sejour(description='Ile de Serifos, dans les Cyclades en Grèce', prix=580, lieux='Grèce/LesCyclades', duree=8, diff='Découverte', theme='Rando nature'),
 Sejour(description="Belle Île en Mer lors d'une randonnée", prix=580, lieux='Bretagne/Belle-IleenMer', duree=6, diff='Initié', theme='Rando nature'),
 Sejour(description='Randonnée dans les Cinque Terre', prix=715, lieux='Italie/CinqueTerre', duree=6, diff='Découverte', theme='Rando nature'),
 Sejour(description='Canaries, r

In [234]:
with open("backup_decathlon.json", "w") as fichier:
    fichier.write(to_json(resultats))